In [2]:
import praw
from kafka import KafkaProducer
from json import dumps, loads
import json

reddit = praw.Reddit(
    client_id="VYwI_9Xqdf4-j6YbAcIXCA",
    client_secret="OvQUVB1QMNs0Xuo9tkOQ9BqVxH-Kmg",
    password="Billion@99",
    user_agent="my_bigdata",
    username="v1nomad",
)

def fetch_data_from_reddit(subreddit_name, keywords):
    subreddit = reddit.subreddit(subreddit_name)
    for submission in subreddit.new(limit=None):
        if all(keyword.lower() in submission.title.lower() for keyword in keywords):
            post_id = submission.id
            title = submission.title
            url = submission.url
            score = submission.score
            upvotes = submission.ups
            downvotes = submission.downs
            num_comments = submission.num_comments
            text = submission.selftext
            author = submission.author.name if submission.author else None
            author_post_karma = None
            if submission.author:
                author_info = reddit.redditor(submission.author.name)
                author_post_karma = author_info.link_karma
            tag = submission.link_flair_text if submission.link_flair_text else None
            comments_data = []
            for comment in submission.comments:
                if isinstance(comment, praw.models.MoreComments):
                    continue  # Skip MoreComments objects
                comment_data = {
                    'comment_id': comment.id,
                    'author': comment.author.name if comment.author else None,
                    'datetime': comment.created_utc,
                    'text': comment.body
                }
                comments_data.append(comment_data)

            yield {
                'post_id': post_id,
                'title': title,
                'url': url,
                'score': score,
                'upvotes': upvotes,
                'downvotes': downvotes,
                'num_comments': num_comments,
                'text': text,
                'author': author,
                'author_post_karma': author_post_karma,
                'tag': tag,
                'comments': comments_data
            }

def kafka_producer(subreddit_name, keywords):
    producer = KafkaProducer(bootstrap_servers=['18.234.36.200:9092'])
    for data in fetch_data_from_reddit(subreddit_name, keywords):
        serialized_data = json.dumps(data).encode('utf-8')
        producer.send('technot', serialized_data)
    producer.flush()

if __name__ == "__main__":
    subreddit_name = 'technology'
    keywords_input = input("Enter keywords separated by commas: ")
    keywords = [keyword.strip() for keyword in keywords_input.split(",")]
    kafka_producer(subreddit_name, keywords)


Enter keywords separated by commas: apple,vision,pro
